In [ ]:
#LCEL - using langchain expression language(lcel) to chain components together
#deploying application using langserve


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Set environment variables for LangSmith tracking and LangChain project
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("SIMPLE_LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
#open ai key
import openai
openai.api_key=os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [ ]:
#for opensource models --llama3, gemma 2,mistral etc we will use groq, groq helps to access all open source models acts as wrappers
from langchain_ollama import OllamaLLM
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

model = ChatGroq(model="gemma2-9b-it",groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021FF12EF6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021FF131E0B0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
#system message - to set the context of the conversation
#human message - actual user input
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French.Translate the following from english to french"),
    HumanMessage(content="Hello, How are you. I love programming.")
]

result = model.invoke(messages)

In [12]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(result)

"Here is the translation:\n\n**Hello, How are you? I love programming.**\n\n**Bonjour, comment allez-vous ? J'adore programmer.** \n\n\nLet me know if you have anything else you'd like to translate! 😊\n"

In [14]:
#using LCEL - langchain expression language to chain components together
chain=model|output_parser
chain.invoke(messages)

"Here's the translation:\n\n**Bonjour, comment allez-vous ? J'adore programmer.** \n"

In [16]:
#prompt template - to create dynamic prompts
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]   

)

In [19]:
result = prompt.invoke({"language":"French","text":"Hello, How are you."})

In [20]:
result.to_messages()


[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, How are you.', additional_kwargs={}, response_metadata={})]

In [21]:
chain=prompt|model|output_parser #using lcel prompt,model,output parser are combined together
chain.invoke({"language":"French","text":"Hello, How are you."})

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal: \n\n**Salut, ça va ?** \n'